# Problem Statement:

A US-based housing company named Surprise Housing has decided to enter the Australian market. The company uses data analytics to purchase houses at a price below their actual values and flip them on at a higher price. For the same purpose, the company has collected a data set from the sale of houses in Australia. The data is provided in the CSV file below.

 

The company is looking at prospective properties to buy to enter the market. You are required to build a regression model using regularisation in order to predict the actual value of the prospective properties and decide whether to invest in them or not.

 

The company wants to know the following things about the prospective properties:

Which variables are significant in predicting the price of a house, and

How well those variables describe the price of a house.

 

Also, determine the optimal value of lambda for ridge and lasso regression.

In [2]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
import statsmodels
import statsmodels.api as sm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

In [3]:
price = pd.read_csv("train.csv")

In [4]:
price.shape

(1460, 81)

In [5]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [9]:
null_perc = round(price.isnull().mean()*100,2)
null_perc   

# The null percentage of columns tells us which are the columns to drop immediatly.

Id                0.00
MSSubClass        0.00
MSZoning          0.00
LotFrontage      17.74
LotArea           0.00
                 ...  
MoSold            0.00
YrSold            0.00
SaleType          0.00
SaleCondition     0.00
SalePrice         0.00
Length: 81, dtype: float64

In [11]:
# Lets see the columns with null percentage greater than 45% and drop them immediatly,
# as high null value percentage won't help our final model. 

null_perc[null_perc > 45]

Alley          93.77
FireplaceQu    47.26
PoolQC         99.52
Fence          80.75
MiscFeature    96.30
dtype: float64

In [12]:
price= price.drop(['FireplaceQu', 'Alley', 'PoolQC', 'Fence', 'MiscFeature'], 1)

In [20]:
null_perc = round(price.isnull().mean()*100,2)
null_perc[null_perc != 0]

LotFrontage     17.74
MasVnrType       0.55
MasVnrArea       0.55
BsmtQual         2.53
BsmtCond         2.53
BsmtExposure     2.60
BsmtFinType1     2.53
BsmtFinType2     2.60
Electrical       0.07
GarageType       5.55
GarageYrBlt      5.55
GarageFinish     5.55
GarageQual       5.55
GarageCond       5.55
dtype: float64

In [22]:
price.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [23]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [37]:
cat_vars = price.select_dtypes(include = 'object')

In [38]:
cat_vars.columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')

In [40]:
num_vars = price.select_dtypes(include = ['int64', 'float64'])

In [41]:
num_vars.columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [43]:
cat_vars.apply(pd.Series.value_counts)


,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
1.5Fin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.5Unf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1Fam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1Story,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.5Fin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WdShake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WdShing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WdShngl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
for col in cat_vars.columns:
    print('-' * 40 + col + '-' * 40 , end=' - ')
    display(cat_vars[col].value_counts())

----------------------------------------MSZoning---------------------------------------- - 

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

----------------------------------------Street---------------------------------------- - 

Pave    1454
Grvl       6
Name: Street, dtype: int64

----------------------------------------LotShape---------------------------------------- - 

...

Y    1340
N      90
P      30
Name: PavedDrive, dtype: int64

----------------------------------------SaleType---------------------------------------- - 

WD       1267
New       122
COD        43
ConLD       9
ConLw       5
ConLI       5
CWD         4
Oth         3
Con         2
Name: SaleType, dtype: int64

----------------------------------------SaleCondition---------------------------------------- - 

Normal     1198
Partial     125
Abnorml     101
Family       20
Alloca       12
AdjLand       4
Name: SaleCondition, dtype: int64

In [48]:
for col in cat_vars.columns:
    print('-' * 40 + col + '-' * 40 , end=' - ')
    display(cat_vars[col].value_counts(normalize = True))

----------------------------------------MSZoning---------------------------------------- - 

RL         0.788356
RM         0.149315
FV         0.044521
RH         0.010959
C (all)    0.006849
Name: MSZoning, dtype: float64

----------------------------------------Street---------------------------------------- - 

Pave    0.99589
Grvl    0.00411
Name: Street, dtype: float64

----------------------------------------LotShape---------------------------------------- - 

...

Y    0.917808
N    0.061644
P    0.020548
Name: PavedDrive, dtype: float64

----------------------------------------SaleType---------------------------------------- - 

WD       0.867808
New      0.083562
COD      0.029452
ConLD    0.006164
ConLw    0.003425
ConLI    0.003425
CWD      0.002740
Oth      0.002055
Con      0.001370
Name: SaleType, dtype: float64

----------------------------------------SaleCondition---------------------------------------- - 

Normal     0.820548
Partial    0.085616
Abnorml    0.069178
Family     0.013699
Alloca     0.008219
AdjLand    0.002740
Name: SaleCondition, dtype: float64

In [50]:
skewed_vars = cat_vars[['Street', 'LandContour', 'Utilities', 'LandSlope', 'Condition1', 'Condition2', 
                        'RoofMatl', 'ExterCond', 'BsmtCond', 'BsmtFinType2', 'Heating', 'CentralAir', 
                        'Electrical','Functional', 'GarageQual', 'GarageCond', 'PavedDrive','SaleType'] ]   

In [51]:
price = price.drop(skewed_vars, 1)

In [52]:
cat_vars = price.select_dtypes(include = 'object')
cat_vars.columns

Index(['MSZoning', 'LotShape', 'LotConfig', 'Neighborhood', 'BldgType',
       'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1',
       'HeatingQC', 'KitchenQual', 'GarageType', 'GarageFinish',
       'SaleCondition'],
      dtype='object')

In [53]:
for col in cat_vars.columns:
    print('-' * 40 + col + '-' * 40 , end=' - ')
    display(cat_vars[col].value_counts(normalize = True))

----------------------------------------MSZoning---------------------------------------- - 

RL         0.788356
RM         0.149315
FV         0.044521
RH         0.010959
C (all)    0.006849
Name: MSZoning, dtype: float64

----------------------------------------LotShape---------------------------------------- - 

Reg    0.633562
IR1    0.331507
IR2    0.028082
IR3    0.006849
Name: LotShape, dtype: float64

----------------------------------------LotConfig---------------------------------------- - 

...

Attchd     0.630892
Detchd     0.280638
BuiltIn    0.063814
Basment    0.013778
CarPort    0.006526
2Types     0.004351
Name: GarageType, dtype: float64

----------------------------------------GarageFinish---------------------------------------- - 

Unf    0.438724
RFn    0.306019
Fin    0.255257
Name: GarageFinish, dtype: float64

----------------------------------------SaleCondition---------------------------------------- - 

Normal     0.820548
Partial    0.085616
Abnorml    0.069178
Family     0.013699
Alloca     0.008219
AdjLand    0.002740
Name: SaleCondition, dtype: float64

In [54]:
cat_vars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   LotShape       1460 non-null   object
 2   LotConfig      1460 non-null   object
 3   Neighborhood   1460 non-null   object
 4   BldgType       1460 non-null   object
 5   HouseStyle     1460 non-null   object
 6   RoofStyle      1460 non-null   object
 7   Exterior1st    1460 non-null   object
 8   Exterior2nd    1460 non-null   object
 9   MasVnrType     1452 non-null   object
 10  ExterQual      1460 non-null   object
 11  Foundation     1460 non-null   object
 12  BsmtQual       1423 non-null   object
 13  BsmtExposure   1422 non-null   object
 14  BsmtFinType1   1423 non-null   object
 15  HeatingQC      1460 non-null   object
 16  KitchenQual    1460 non-null   object
 17  GarageType     1379 non-null   object
 18  GarageFinish   1379 non-null

In [58]:
cat_vars.isnull().sum()

MSZoning          0
LotShape          0
LotConfig         0
Neighborhood      0
BldgType          0
HouseStyle        0
RoofStyle         0
Exterior1st       0
Exterior2nd       0
MasVnrType        8
ExterQual         0
Foundation        0
BsmtQual         37
BsmtExposure     38
BsmtFinType1     37
HeatingQC         0
KitchenQual       0
GarageType       81
GarageFinish     81
SaleCondition     0
dtype: int64

In [59]:
# Fill the NAN cells with mode.
price.MasVnrType = price.MasVnrType.fillna(price.MasVnrType.mode()[0])
price.BsmtQual = price.BsmtQual.fillna(price.BsmtQual.mode()[0])
price.BsmtExposure = price.BsmtExposure.fillna(price.BsmtExposure.mode()[0])
price.BsmtFinType1 = price.BsmtFinType1.fillna(price.BsmtFinType1.mode()[0])
price.GarageType = price.GarageType.fillna(price.GarageType.mode()[0])
price.GarageFinish = price.GarageFinish.fillna(price.GarageFinish.mode()[0])

In [61]:
price[cat_vars.columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   LotShape       1460 non-null   object
 2   LotConfig      1460 non-null   object
 3   Neighborhood   1460 non-null   object
 4   BldgType       1460 non-null   object
 5   HouseStyle     1460 non-null   object
 6   RoofStyle      1460 non-null   object
 7   Exterior1st    1460 non-null   object
 8   Exterior2nd    1460 non-null   object
 9   MasVnrType     1460 non-null   object
 10  ExterQual      1460 non-null   object
 11  Foundation     1460 non-null   object
 12  BsmtQual       1460 non-null   object
 13  BsmtExposure   1460 non-null   object
 14  BsmtFinType1   1460 non-null   object
 15  HeatingQC      1460 non-null   object
 16  KitchenQual    1460 non-null   object
 17  GarageType     1460 non-null   object
 18  GarageFinish   1460 non-null

In [62]:
cat_vars = price.select_dtypes(include = 'object')

In [64]:
cat_vars.isnull().sum()

MSZoning         0
LotShape         0
LotConfig        0
Neighborhood     0
BldgType         0
HouseStyle       0
RoofStyle        0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
Foundation       0
BsmtQual         0
BsmtExposure     0
BsmtFinType1     0
HeatingQC        0
KitchenQual      0
GarageType       0
GarageFinish     0
SaleCondition    0
dtype: int64

In [65]:
for col in cat_vars.columns:
    print('-' * 40 + col + '-' * 40 , end=' - ')
    display(cat_vars[col].value_counts(normalize = True))

----------------------------------------MSZoning---------------------------------------- - 

RL         0.788356
RM         0.149315
FV         0.044521
RH         0.010959
C (all)    0.006849
Name: MSZoning, dtype: float64

----------------------------------------LotShape---------------------------------------- - 

Reg    0.633562
IR1    0.331507
IR2    0.028082
IR3    0.006849
Name: LotShape, dtype: float64

----------------------------------------LotConfig---------------------------------------- - 

...

Attchd     0.651370
Detchd     0.265068
BuiltIn    0.060274
Basment    0.013014
CarPort    0.006164
2Types     0.004110
Name: GarageType, dtype: float64

----------------------------------------GarageFinish---------------------------------------- - 

Unf    0.469863
RFn    0.289041
Fin    0.241096
Name: GarageFinish, dtype: float64

----------------------------------------SaleCondition---------------------------------------- - 

Normal     0.820548
Partial    0.085616
Abnorml    0.069178
Family     0.013699
Alloca     0.008219
AdjLand    0.002740
Name: SaleCondition, dtype: float64

In [66]:
# Define the changes to be made to 'Last Notable Activity' column and apply the changes
def Change_MSZoning(x):
    if x == 'RL':
        return 'RL'
    elif x == 'RM':
        return 'RM'
    else: 
        return 'Others'

price.MSZoning = price.MSZoning.apply(Change_MSZoning)

In [67]:
# Define the changes to be made to 'Last Notable Activity' column and apply the changes
def Change_LotShape(x):
    if x == 'Reg':
        return 'Reg'
    elif x == 'IR1':
        return 'IR1'
    else: 
        return 'Others'

price.LotShape = price.LotShape.apply(Change_LotShape)

In [68]:
# Define the changes to be made to 'Last Notable Activity' column and apply the changes
def Change_Neighborhood(x):
    if x == 'NAmes':
        return 'NAmes'
    elif x == 'CollgCr':
        return 'CollgCr'
    elif x == 'OldTown':
        return 'OldTown'
    elif x == 'Edwards':
        return 'Edwards'
    else: 
        return 'Others'

price.Neighborhood = price.Neighborhood.apply(Change_Neighborhood)

In [69]:
# Define the changes to be made to 'Last Notable Activity' column and apply the changes
def Change_BldgType(x):
    if x == '1Fam':
        return '1Fam'
    elif x == 'TwnhsE':
        return 'TwnhsE'
    else: 
        return 'Others'

price.BldgType = price.BldgType.apply(Change_BldgType)


In [70]:
# Define the changes to be made to 'Last Notable Activity' column and apply the changes
def Change_HouseStyle(x):
    if x == '1Story':
        return '1Story'
    elif x == '2Story':
        return '2Story'
    elif x == '1.5Fin':
        return '1.5Fin'
    else: 
        return 'Others'

price.HouseStyle = price.HouseStyle.apply(Change_HouseStyle)

In [72]:

def Change_RoofStyle(x):
    if x == 'Gable':
        return 'Gable'
    elif x == 'Hip':
        return 'Hip'
    else: 
        return 'Others'

def Change_Exterior1st(x):
    if x == 'VinylSd':
        return 'VinylSd'
    elif x == 'HdBoard':
        return 'HdBoard'
    elif x == 'MetalSd':
        return 'MetalSd'
    elif x == 'Wd Sdng':
        return 'WdSdng'
    else: 
        return 'Others'

def Change_ExterQual(x):
    if x == 'TA':
        return 'TA'
    elif x == 'Gd':
        return 'Gd'
    else: 
        return 'Others'

def Change_Foundation(x):
    if x == 'PConc':
        return 'PConc'
    elif x == 'CBlock':
        return 'CBlock'
    else: 
        return 'Others'

def Change_BsmtQual(x):
    if x == 'TA':
        return 'TA'
    elif x == 'Gd':
        return 'Gd'
    else: 
        return 'Others'

def Change_HeatingQC(x):
    if x == 'Ex':
        return 'Ex'
    elif x == 'TA':
        return 'TA'
    elif x == 'Gd':
        return 'Gd'
    else: 
        return 'Others'

def Change_KitchenQual(x):
    if x == 'TA':
        return 'TA'
    elif x == 'Gd':
        return 'Gd'
    else: 
        return 'Others'

def Change_GarageType(x):
    if x == 'Attchd':
        return 'Attchd'
    elif x == 'Detchd':
        return 'Detchd'
    else: 
        return 'Others'

def Change_SaleCondition(x):
    if x == 'Normal':
        return 'Normal'
    elif x == 'Partial':
        return 'Partial'
    else: 
        return 'Others'



price.RoofStyle = price.RoofStyle.apply(Change_RoofStyle)
price.Exterior1st = price.Exterior1st.apply(Change_Exterior1st)
price.ExterQual = price.ExterQual.apply(Change_ExterQual)
price.Foundation = price.Foundation.apply(Change_Foundation)
price.BsmtQual = price.BsmtQual.apply(Change_BsmtQual)
price.HeatingQC = price.HeatingQC.apply(Change_HeatingQC)
price.KitchenQual = price.KitchenQual.apply(Change_KitchenQual)
price.GarageType = price.GarageType.apply(Change_GarageType)
price.SaleCondition = price.SaleCondition.apply(Change_SaleCondition)




In [75]:

def Change_Exterior2nd(x):
    if x == 'VinylSd':
        return 'VinylSd'
    elif x == 'HdBoard':
        return 'HdBoard'
    elif x == 'MetalSd':
        return 'MetalSd'
    elif x == 'Wd Sdng':
        return 'WdSdng'
    elif x == 'Plywood':
        return 'Plywood'
    else: 
        return 'Others'
    
price.Exterior2nd = price.Exterior2nd.apply(Change_Exterior2nd)


In [77]:
def Change_LotConfig(x):
    if x == 'Inside':
        return 'Inside'
    elif x == 'Corner':
        return 'Corner'
    else: 
        return 'Others'
    
price.LotConfig = price.LotConfig.apply(Change_LotConfig)


In [78]:
cat_vars = price.select_dtypes(include = 'object')

for col in cat_vars.columns:
    print('-' * 40 + col + '-' * 40 , end=' - ')
    display(cat_vars[col].value_counts(normalize = True))

----------------------------------------MSZoning---------------------------------------- - 

RL        0.788356
RM        0.149315
Others    0.062329
Name: MSZoning, dtype: float64

----------------------------------------LotShape---------------------------------------- - 

Reg       0.633562
IR1       0.331507
Others    0.034932
Name: LotShape, dtype: float64

----------------------------------------LotConfig---------------------------------------- - 

...

Attchd    0.651370
Detchd    0.265068
Others    0.083562
Name: GarageType, dtype: float64

----------------------------------------GarageFinish---------------------------------------- - 

Unf    0.469863
RFn    0.289041
Fin    0.241096
Name: GarageFinish, dtype: float64

----------------------------------------SaleCondition---------------------------------------- - 

Normal     0.820548
Others     0.093836
Partial    0.085616
Name: SaleCondition, dtype: float64

In [80]:
cat_vars.columns

Index(['MSZoning', 'LotShape', 'LotConfig', 'Neighborhood', 'BldgType',
       'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual',
       'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC',
       'KitchenQual', 'GarageType', 'GarageFinish', 'SaleCondition'],
      dtype='object')

In [81]:
# Creating a dummy variable for some of the categorical variables and dropping the first one.
dummy1 = pd.get_dummies(price[['MSZoning', 'LotShape', 'LotConfig', 'Neighborhood', 'BldgType',
       'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual',
       'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC',
       'KitchenQual', 'GarageType', 'GarageFinish', 'SaleCondition']], drop_first=True)

# Adding the results to the master dataframe
price = pd.concat([price, dummy1], axis=1)

In [82]:
price.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LotConfig,Neighborhood,BldgType,HouseStyle,...,HeatingQC_Others,HeatingQC_TA,KitchenQual_Others,KitchenQual_TA,GarageType_Detchd,GarageType_Others,GarageFinish_RFn,GarageFinish_Unf,SaleCondition_Others,SaleCondition_Partial
0,1,60,RL,65.0,8450,Reg,Inside,CollgCr,1Fam,2Story,...,0,0,0,0,0,0,1,0,0,0
1,2,20,RL,80.0,9600,Reg,Others,Others,1Fam,1Story,...,0,0,0,1,0,0,1,0,0,0
2,3,60,RL,68.0,11250,IR1,Inside,CollgCr,1Fam,2Story,...,0,0,0,0,0,0,1,0,0,0
3,4,70,RL,60.0,9550,IR1,Corner,Others,1Fam,2Story,...,0,0,0,0,1,0,0,1,1,0
4,5,60,RL,84.0,14260,IR1,Others,Others,1Fam,2Story,...,0,0,0,0,0,0,1,0,0,0


In [83]:
# Drop the categorical columns after creating dummines
price = price.drop(['MSZoning', 'LotShape', 'LotConfig', 'Neighborhood', 'BldgType',
       'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual',
       'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC',
       'KitchenQual', 'GarageType', 'GarageFinish', 'SaleCondition'], axis = 1)

In [84]:
price.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,HeatingQC_Others,HeatingQC_TA,KitchenQual_Others,KitchenQual_TA,GarageType_Detchd,GarageType_Others,GarageFinish_RFn,GarageFinish_Unf,SaleCondition_Others,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,0,0,0,1,0,0,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,1,0,0,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,0,0,0,1,0,0,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,0,1,0,0,1,1,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,0,0,0,1,0,0,0


In [85]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 89 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     1460 non-null   int64  
 1   MSSubClass             1460 non-null   int64  
 2   LotFrontage            1201 non-null   float64
 3   LotArea                1460 non-null   int64  
 4   OverallQual            1460 non-null   int64  
 5   OverallCond            1460 non-null   int64  
 6   YearBuilt              1460 non-null   int64  
 7   YearRemodAdd           1460 non-null   int64  
 8   MasVnrArea             1452 non-null   float64
 9   BsmtFinSF1             1460 non-null   int64  
 10  BsmtFinSF2             1460 non-null   int64  
 11  BsmtUnfSF              1460 non-null   int64  
 12  TotalBsmtSF            1460 non-null   int64  
 13  1stFlrSF               1460 non-null   int64  
 14  2ndFlrSF               1460 non-null   int64  
 15  LowQ